# VNS - Variable Neighborhood Search 

# Leitura de Instâncias

In [2]:
import sys
import time

import numpy as np



class Instance:
    def __init__(self, filename):

        path = f"{filename}"

        f = open(path, "r")

        self.num_items, self.num_forfeits_pairs, self.budget = map(
            int, f.readline().split(" ")
        )

        f.close()

        # items definition
        items = []
        for i in range(self.num_items):
            items.append(i)

        self.items = np.array(items)

        line_counter = 1

        self.profits = np.loadtxt(
            path, delimiter=" ", skiprows=line_counter, max_rows=1
        )
        line_counter += 1
        
        self.weights = np.loadtxt(
            path, delimiter=" ", skiprows=line_counter, max_rows=1
        )
        line_counter += 1

        max = 2 * self.num_forfeits_pairs

        self.forfeit_cost_and_forfeits_pairs = np.loadtxt(
            path, delimiter=" ", skiprows=line_counter, max_rows=max, usecols=(0, 1)
        ).tolist()

        self.forfeits_costs = [
            v for i, v in enumerate(self.forfeit_cost_and_forfeits_pairs) if i % 2 == 0
        ]

        self.forfeits_pairs = [
            v for i, v in enumerate(self.forfeit_cost_and_forfeits_pairs) if i % 2 != 0
        ]

        # remove the first element of the sublist
        for i in range(len(self.forfeits_costs)):
            self.forfeits_costs[i].pop(0)

        # transform list of lists into a single list - flatten
        self.forfeits_costs = [
            item for sublist in self.forfeits_costs for item in sublist
        ]

        self.forfeits_costs = np.array(self.forfeits_costs)
        self.forfeits_pairs = np.array(self.forfeits_pairs)      

# Matriz de Penalidades

In [3]:
def calculate_penalty(items,forfeits_pairs, forfeits_costs):
    mD=np.zeros((int(len(items)),int(len(items))))       
    
    for index, pair in enumerate(forfeits_pairs):
        mD[int(pair[0]),int(pair[1])]=forfeits_costs[index]                
    return mD

# Algoritmo Construtivo

In [4]:
import random as rd

def greedyalgorithm(items, weights, profits, budget, forfeits, forfeits_costs, alpha,mD):    
    
    cap=budget    

    #print(sorted_items)    
    solution=[]
    scost=[]
    sweights=[]
    cost=0
    index=0
    
    if alpha == 0:  # totalmente aleatório 
        remaining_items=zip(items,weights,profits)
        sorted_items=sorted(remaining_items, key= lambda x:x[2]/(x[1]+1), reverse=True)
        #print(tuple(items))        
        
        rd_index=rd.choice(range(0,len(sorted_items))) 
        candidate=sorted_items[rd_index][0]
        
        while cap-weights[candidate]>0:                                        
                cap=cap-weights[candidate]
                solution.append(candidate) 
                
                
                penalidade=sum(mD[candidate][solution])                
                #if penalidade>0: print(f"Penalidade do Item {sorted_items[rd_index][0]} é {penalidade}")
                    
                cost=cost+profits[candidate]-penalidade
                
                scost.append(profits[candidate]-penalidade)
                
                sweights.append(weights[candidate])
                
                sorted_items.remove(sorted_items[rd_index])
                rd_index=rd.choice(range(0,len(sorted_items)))  
                candidate=sorted_items[rd_index][0]
        
                    
    else:
            if alpha == 1: #totalmente guloso
                remaining_items=zip(items,weights,profits)
                #print(tuple(items))
                sorted_items=sorted(remaining_items, key= lambda x:x[2]/(x[1]+1), reverse=True)
                #sorted_items=sorted(remaining_items, key= lambda x:x[2]/x[1], reverse=True)
                #print(sorted_items)
                candidate=sorted_items[0][0]
                while cap-weights[candidate]>0:                                                  
                    
                    #calcula a nova penalidade para o item a ser inserido no conjunto e reordena                                      
                    #sorted_items=sorted(sorted_items, key= lambda x:(x[2]/x[1])-sum(mD[x[0]][solution]), reverse=True)                                        
                    
                    cap=cap-weights[candidate]
                    solution.append(candidate)  
                    
                    penalidade=sum(mD[candidate][solution])                
                    #if penalidade>0: print(f"Penalidade do Item {sorted_items[0][0]} é {penalidade}")
                    
                    #cost=cost+sorted_items[0][2]
                    cost=cost+profits[candidate]-penalidade
                    #cost=cost+sorted_items[index][2]-sum(mD[sorted_items[index][0]][solution])
                    
                    scost.append(profits[candidate]-penalidade)
                    
                    sweights.append(weights[candidate])
                    
                    #sorted_items=sorted(sorted_items, key= lambda x:(x[2]-sum(mD[x[0]][solution]))/(x[1]+1), reverse=True)                                        
                    sorted_items.remove(sorted_items[0])
                    
                    if len(sorted_items)>=0:
                        #sorted_items=sorted(sorted_items, key= lambda x:(x[2]-sum(mD[x[0]][solution]))/(x[1]+1), reverse=True)                                        
                        candidate=sorted_items[0][0]
                    #sorted_items.remove(sorted_items[index])
                    #index=index+1  
                    
            else: # semi-guloso
                remaining_items=zip(items,weights,profits,profits/weights)
                sorted_items=sorted(remaining_items, key= lambda x:x[2]/(x[1]+1), reverse=True)
                
                
                # o cálculo dos limites da lcr
                hmax=sorted_items[0][3]
                hmin=sorted_items[-1][3]

                    #ub = hmax + alpha * (hmin - hmax)
                lb = hmin                            
                ub=hmax + alpha * (hmin - hmax)                  

                    #lista restrita de candidatos
                    #lcr=list(filter(lambda x: x[3]<=ub and x[3] >= lb, sorted_items))                                
                lcr=list(filter(lambda x: x[3]>=ub, sorted_items))
                rd_index=rd.choice(range(0,len(lcr))) 
                candidate=lcr[rd_index][0]
                
                #print(candidate)
                while cap-weights[candidate]>0:  
                                                                          
                    cap=cap-weights[candidate]
                    solution.append(candidate)    
                    #print(f" Solution: {solution}")
                                        
                    
                    #penalidade
                    penalidade=sum(mD[candidate][solution])                
                    #if penalidade>0: print(f"Penalidade do Item {sorted_items[rd_index][0]} é {penalidade}")
                                        
                    cost=cost+profits[candidate]-penalidade                                        
                    
                    scost.append(profits[candidate]-penalidade)
                    
                    sweights.append(weights[candidate])
                
                    #removendo o item já inserido
                    sorted_items.remove(sorted_items[sorted_items.index(lcr[rd_index])])                    
               
                                          
                    #sorted_items.sort(key= lambda x:(x[2]-sum(mD[x[0]][solution]))/(x[1]+1), reverse=True)                                        

                     # o cálculo dos limites da lcr
                    hmax=sorted_items[0][3]
                    hmin=sorted_items[-1][3]
                    
                    #ub = hmax + alpha * (hmin - hmax)
                    lb = hmin                            
                    ub=hmax + alpha * (hmin - hmax)
                 
                    #lista restrita de candidatos                                
                    lcr=list(filter(lambda x: x[3]>=ub, sorted_items))

                    #print(f"LCR: {lcr}")
                    rd_index=rd.choice(range(0,len(lcr))) 
                    candidate=lcr[rd_index][0]
                                
    
    return solution,cost,cap,scost,sweights,sorted_items

# VNS

## Local searches structures

First Neighborhood Structure

In [5]:
import random

def get_solution_cost(solution, profits, forfeits_pairs_set, mD) -> int:
  cost = 0

  for item in solution:
      cost += profits[item]

  for i in solution:
    for j in solution:
      if i > j:
        if (i, j) in forfeits_pairs_set:
          cost -= mD[i][j]

  return cost


def swap(solution:set, enter_item:int, leave_item:int) -> set:
  solution.remove(leave_item)
  solution.add(enter_item)

  return solution



def local_search_k1(solution, items, weights, profits, solution_cost, budget, forfeits_costs, forfeits_pairs, mD):
    solution_set = set(solution)
    items_set = set(items)

    # convert forfeits pair in a forfeits pair set
    forfeits_pairs_set = {tuple(x) for x in forfeits_pairs}

    improved = True
    ls_cost = int()

    # print(f'constructive solution cost: {solution_cost}')

    # solution_cost = get_solution_cost(solution_set, profits, forfeits_pairs_set, mD)
    # print(f'busca local solution cost: {solution_cost}')
    best_improvement = solution_cost
    # print(f'initial_solution_cost:{solution_cost}')

    while improved == True:

      improved = False

      # choosing a random element to enter in solution
      # enter_item = random.choice(tuple(remaining_items)) -> antes o item era escolhida aleatoriamente na lista de restantes

      solution_cost = get_solution_cost(solution_set, profits, forfeits_pairs_set, mD)
      # print(f'get_solution_cost:{solution_cost}')
      cost = solution_cost


      # items not in solution yet
      remaining_items = items_set.difference(solution_set)

      solution_weights = 0
      for item in solution_set:
        solution_weights += weights[item]

      # print(f"solution_weights:{solution_weights}")
      # print(f"budget:{budget}")
      temp_weight = 0

      for sol_item in solution_set: # será iterado apenas uma vez por todos itens que não estão na solução
        for item in remaining_items:
            
          # enter_item = item
          # leave_item = sol_item

          temp_weight = solution_weights - weights[sol_item] + weights[item]
          # print(f'temp_weight:{temp_weight}')

          if temp_weight <= budget: #feasibility
            ls_cost = cost - profits[sol_item] + profits[item]

            for i in solution_set: # add penalties of leave item
              if (i, sol_item) in forfeits_pairs_set:
                  ls_cost += mD[i][sol_item]

              if (item, sol_item) in forfeits_pairs_set:
                  ls_cost += mD[item][i]

            for i in solution_set: # subtract penalties of enter item
              if (i, item) in forfeits_pairs_set:
                  ls_cost -= mD[i][item]

              if (i, item) in forfeits_pairs_set:
                  ls_cost -= mD[i][item]

            if best_improvement < ls_cost:
              best_improvement = ls_cost
              enter_item = item
              leave_item = sol_item
              improved = True
              print(f'improved: {best_improvement}')

      if improved == True:
        solution_set = swap(solution_set, enter_item, leave_item)    

    return best_improvement, list(solution_set)

Second Neighborhood Structure

In [6]:
import random

def get_solution_cost(solution, profits, forfeits_pairs_set, mD) -> int:
  cost = 0

  for item in solution:
      cost += profits[item]

  for i in solution:
    for j in solution:
      if i > j:
        if (i, j) in forfeits_pairs_set or (j,i) in forfeits_pairs_set:
          cost -= mD[i][j]

  return cost


def swap2(solution:set, enter_item:int, leave_item:int, weights, budget, items) -> set:
  solution.remove(leave_item)
  solution.add(enter_item)

  # code to remove the second element
  #remove a random feasible second element
  solution_weights = 0
  for item in solution:
    solution_weights += weights[item]

  # items not in solution yet
  items_set = set(items)
  remaining_items = items_set.difference(solution)
    
  # choosing a random element to enter in solution
  enter_item2 = random.choice(tuple(remaining_items))

  # choosing a random element to leave the solution
  leave_item2 = random.choice(tuple(solution))

  temp_weight = solution_weights - weights[leave_item2] + weights[enter_item2]

  if temp_weight <= budget: #feasibility
        solution.remove(leave_item2)
        solution.add(enter_item2)

  return solution


def local_search_k2(solution, items, weights, profits, solution_cost, budget, forfeits_costs, forfeits_pairs, mD):

    solution_set = set(solution)
    items_set = set(items)

    # convert forfeits pair in a forfeits pair set
    forfeits_pairs_set = {tuple(x) for x in forfeits_pairs}

    improved = True
    ls_cost = int()

    # print(f'constructive solution cost: {solution_cost}')

    # solution_cost = get_solution_cost(solution_set, profits, forfeits_pairs_set, mD)
    # print(f'busca local solution cost: {solution_cost}')
    best_improvement = solution_cost
    # print(f'initial_solution_cost:{solution_cost}')

    while improved == True:

      improved = False

      # choosing a random element to enter in solution
      # enter_item = random.choice(tuple(remaining_items)) -> antes o item era escolhida aleatoriamente na lista de restantes

      solution_cost = get_solution_cost(solution_set, profits, forfeits_pairs_set, mD)
      # print(f'get_solution_cost:{solution_cost}')
      cost = solution_cost

      # items not in solution yet
      remaining_items = items_set.difference(solution_set)

      solution_weights = 0
      for item in solution_set:
        solution_weights += weights[item]

      # print(f"solution_weights:{solution_weights}")
      # print(f"budget:{budget}")
      temp_weight = 0

      for sol_item in solution_set: # será iterado apenas uma vez por todos itens que não estão na solução
        for item in remaining_items:
            
          # enter_item = item
          # leave_item = sol_item

          temp_weight = solution_weights - weights[sol_item] + weights[item]
          # print(f'temp_weight:{temp_weight}')

          if temp_weight <= budget: #feasibility
            ls_cost = cost - profits[sol_item] + profits[item]

            for i in solution_set: # add penalties of leave item
              if (i, sol_item) in forfeits_pairs_set:
                  ls_cost += mD[i][sol_item]

              if (item, sol_item) in forfeits_pairs_set:
                  ls_cost += mD[item][i]

            for i in solution_set: # subtract penalties of enter item
              if (i, item) in forfeits_pairs_set:
                  ls_cost -= mD[i][item]

              if (i, item) in forfeits_pairs_set:
                  ls_cost -= mD[i][item]

            if best_improvement < ls_cost:
              best_improvement = ls_cost
              enter_item = item
              leave_item = sol_item
              improved = True
              print(f'improved: {best_improvement}')

      if improved == True:
        solution_set = swap2(solution_set, enter_item, leave_item, weights, budget, items)    

    return best_improvement, list(solution_set)

## VNS definition functions

In [9]:
def get_solution_cost(solution, profits, forfeits_pairs_set, mD) -> int:
  cost = 0

  for item in solution:
      cost += profits[item]

  for i in solution:
    for j in solution:
      if i > j:
        if (i, j) in forfeits_pairs_set:
          cost -= mD[i][j]

  return cost

def neighborhood_change(cost2, cost, s_initial, s_local_search, k):

    local_search_cost = cost2
    initial_cost = cost
    solution = s_initial

    if k == 1:
        if local_search_cost > initial_cost:
            solution = s_local_search
            cost = local_search_cost
            k = 1
        else:
            k = k + 1
    elif k == 2:
        if local_search_cost > initial_cost:
            solution = s_local_search
            cost = local_search_cost
            k = 2
        else:
            k = k + 1  # if this line doesn't exist kmax will never reach an end

    return solution, k, cost



def shake(solution:list(), items, weights, budget) -> list():
    # transform solution list to a set
    solution_set = set(solution)
    items = set(items)

    # items not in solution yet
    remaining_items = items.difference(solution_set)

    for i in range(1,101): # change 10 random numbers
        # choosing a random element to enter in solution
        enter_item = random.choice(tuple(remaining_items))
        # print(f'enter:{enter_item}')
        
        # choosing a random element to leave the solution
        leave_item = random.choice(tuple(solution_set))
        # print(f'leave:{leave_item}')

        solution_weights = 0
        for item in solution_set:
          solution_weights += weights[item]

        temp_weight = solution_weights - weights[leave_item] + weights[enter_item]

        if temp_weight <= budget: #feasibility

          # exchange elements
          solution_set.remove(leave_item)
          solution_set.add(enter_item)

    return list(solution_set)

def VNS(solution, cost, kmax, tmax, items, weights, profits, budget, forfeits_costs, forfeits_pairs, mD):

  iter = 1
  best_cost = cost
  s = solution

  # convert forfeits pair in a forfeits pair set
  forfeits_pairs_set = {tuple(x) for x in forfeits_pairs}

  timeout_start = time.time()
  count_iter = 0
  while time.time() < timeout_start + tmax:
    print(f'iteration: {count_iter}')
    k = 1

    while k <= kmax:
      print(f'neighborhood {k}')
      
      s1 = shake(s, items, weights, budget)

      if k == 1:
        cost2, s2 = local_search_k1(s1, items, weights, profits, cost, budget, forfeits_costs, forfeits_pairs, mD)
      elif k == 2:
        cost2, s2 = local_search_k2(s1, items, weights, profits, cost, budget, forfeits_costs, forfeits_pairs, mD)

      if cost2 > best_cost: # acceptance criterion
        best_cost = cost2
        print(f'melhorou: {best_cost}')

      s, k, cost = neighborhood_change(cost2, cost, s, s2, k) # posso subir os costs diretamente e quebrar logo o loop

    count_iter += 1

  # testar as melhores soluções entre as vizinhanças
  return s, best_cost
  

## Teste VNS

In [11]:
pasta = 'instances/'
filename = pasta+"500/kpf_1.txt"
print(filename)
problem_instance = Instance(filename)

mD=calculate_penalty(problem_instance.items,problem_instance.forfeits_pairs, problem_instance.forfeits_costs)
start_time = time.time()
alpha=1 #guloso

start_time = time.time()

solution_greedy,cost_greedy,cap,scost,sweights,sorted_items = greedyalgorithm(
    problem_instance.items,
    problem_instance.weights,
    problem_instance.profits,
    problem_instance.budget,
    problem_instance.forfeits_costs,
    problem_instance.forfeits_pairs,
    alpha,
    mD,
)

print(f"Custo construtivo: {cost_greedy}")
print(f"Solução construtivo: {solution_greedy}")

neighborhood_structures = 2
tmax = 100 # max execution time in seconds 

solution_vns, cost_vns = VNS(
    solution_greedy,
    cost_greedy,
    neighborhood_structures,
    tmax,
    problem_instance.items, 
    problem_instance.weights,
    problem_instance.profits,
    problem_instance.budget,
    problem_instance.forfeits_costs,
    problem_instance.forfeits_pairs,
    mD)

end_time = time.time()

wall_time = end_time - start_time


print(wall_time)
print(f"Custo VNS: {cost_vns}")
print(f"Solução {solution_vns}")

solution_weights = 0
for item in solution_vns:
  solution_weights += problem_instance.weights[item]
print(f"Solution weights: {solution_weights}")

instances/500/kpf_1.txt
Custo construtivo: 1767.0
Solução construtivo: [271, 192, 72, 159, 196, 471, 77, 174, 456, 90, 116, 478, 130, 40, 63, 187, 66, 310, 346, 95, 353, 406, 463, 22, 99, 241, 296, 446, 464, 5, 173, 204, 450, 14, 219, 257, 295, 186, 481, 17, 79, 117, 211, 300, 402, 145, 135, 170, 317, 482, 491, 134, 473, 127, 290, 21, 39, 87, 171, 229, 253, 488, 480, 161, 227, 239, 256, 309, 362, 367, 457, 476, 44, 101, 427, 495, 38, 226, 337, 382, 311, 314, 18, 224, 37, 163, 254, 73, 356, 202, 234, 380, 327, 19, 265, 54, 150, 262, 364, 395, 359, 425, 9, 59, 82, 178, 220, 92, 247, 368, 409, 315, 89, 370, 151, 423, 140, 329, 412, 35, 93, 96, 111, 142, 162, 177, 237, 266, 297, 344, 366, 373, 378, 389, 403, 413, 440, 454, 240, 312, 8, 455, 42, 328, 434, 91, 109, 225, 363, 47, 11, 209, 55, 246, 264, 339, 386, 60, 188, 245, 181, 199, 484, 4, 396, 437, 128, 182, 269, 164, 102, 213, 288, 348, 411, 486, 16, 29, 477, 417, 452, 154, 157, 138, 41, 146, 203, 201, 52, 56, 94, 175, 248, 305, 306, 41